In [3]:
import requests

# API endpoint
url = "http://127.0.0.1:8000/find-image/"

# Request payload
payload = {
    "description": "A silver Tesla Cybertruck on the highway",  # Replace with your desired description
    "num_images": 5  # Number of images to analyze
}

# Request headers
headers = {
    "Content-Type": "application/json"
}

# Make the POST request
response = requests.post(url, json=payload, headers=headers)


In [ ]:

# Check the response
if response.status_code == 200:
    print("Selected Image Index:", response.json())
else:
    print("Error:", response.status_code, response.text)
